In [141]:
import pandas as pd

In [142]:
all_shots = pd.read_csv("shots.csv")

In [143]:
all_shots

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,21500014,20,202083,Wesley Matthews,1610612742,Dallas Mavericks,1,9,4,...,Right Side Center(RC),24+ ft.,24,78,228,1,0,20151028,PHX,DAL
1,Shot Chart Detail,21500014,168,202083,Wesley Matthews,1610612742,Dallas Mavericks,2,11,2,...,Left Side Center(LC),24+ ft.,25,-232,106,1,0,20151028,PHX,DAL
2,Shot Chart Detail,21500014,217,202083,Wesley Matthews,1610612742,Dallas Mavericks,2,8,10,...,Right Side Center(RC),24+ ft.,24,177,174,1,1,20151028,PHX,DAL
3,Shot Chart Detail,21500014,340,202083,Wesley Matthews,1610612742,Dallas Mavericks,3,10,57,...,Right Side Center(RC),24+ ft.,24,222,111,1,1,20151028,PHX,DAL
4,Shot Chart Detail,21500014,358,202083,Wesley Matthews,1610612742,Dallas Mavericks,3,9,33,...,Right Side(R),16-24 ft.,21,182,110,1,0,20151028,PHX,DAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205533,Shot Chart Detail,21501194,66,202693,Markieff Morris,1610612764,Washington Wizards,1,4,24,...,Center(C),Less Than 8 ft.,2,-19,-16,1,1,20160410,WAS,CHA
205534,Shot Chart Detail,21501194,81,202693,Markieff Morris,1610612764,Washington Wizards,1,3,20,...,Center(C),8-16 ft.,9,-29,95,1,1,20160410,WAS,CHA
205535,Shot Chart Detail,21501194,301,202693,Markieff Morris,1610612764,Washington Wizards,3,10,18,...,Center(C),24+ ft.,27,-60,267,1,0,20160410,WAS,CHA
205536,Shot Chart Detail,21501194,327,202693,Markieff Morris,1610612764,Washington Wizards,3,8,13,...,Center(C),16-24 ft.,18,-17,183,1,1,20160410,WAS,CHA


In [144]:
# Convert shot type to numeric shot value
all_shots["SHOT_VALUE"] = all_shots["SHOT_TYPE"].apply(lambda x: 3 if "3PT" in x else 2)

# Get time to end of period in seconds
all_shots["TIME_LEFT_SEC"] = all_shots["MINUTES_REMAINING"] * 60 + all_shots["SECONDS_REMAINING"]

all_shots = all_shots.drop(columns=["MINUTES_REMAINING", "SECONDS_REMAINING", "SHOT_TYPE"], errors = 'ignore')

# Removing variables
cols_to_drop_round1 = [
    "GRID_TYPE",
    "SHOT_ATTEMPTED_FLAG",
    "PLAYER_ID",
    "TEAM_ID",
    "EVENT_TYPE",
    "GAME_DATE" # distance and region is more valuable than x and y locations, much simpler, dont have to consider side changes
]

# Removing redundant variables and those which would be too nuanced to fit into a coaching plan 

all_shots = all_shots.drop(columns=cols_to_drop_round1, errors='ignore')

all_shots_renamed = all_shots

for col in all_shots.columns:
    print(col)

GAME_ID
GAME_EVENT_ID
PLAYER_NAME
TEAM_NAME
PERIOD
ACTION_TYPE
SHOT_ZONE_BASIC
SHOT_ZONE_AREA
SHOT_ZONE_RANGE
SHOT_DISTANCE
LOC_X
LOC_Y
SHOT_MADE_FLAG
HTM
VTM
SHOT_VALUE
TIME_LEFT_SEC


In [145]:
# Removing variables
cols_to_drop_round2 = [
    "GRID_TYPE",
    "SHOT_ATTEMPTED_FLAG",
    "GAME_ID",
    "GAME_EVENT_ID",
    "PLAYER_ID",
    "TEAM_ID",
    "EVENT_TYPE",
    "GAME_DATE",
    "SHOT_DISTANCE",
    "HTM",
    "VTM",
    "LOC_X",
    "LOC_Y", # distance and region is more valuable than x and y locations, much simpler, dont have to consider side changes
]

# Removing redundant variables and those which would be too nuanced to fit into a coaching plan 

all_shots = all_shots.drop(columns=cols_to_drop_round2, errors='ignore')

In [146]:
allTypes = all_shots

In [147]:
for col in all_shots.columns:
    print(col)

PLAYER_NAME
TEAM_NAME
PERIOD
ACTION_TYPE
SHOT_ZONE_BASIC
SHOT_ZONE_AREA
SHOT_ZONE_RANGE
SHOT_MADE_FLAG
SHOT_VALUE
TIME_LEFT_SEC


In [148]:
# Combine shot zones into one
all_shots['SHOT_ZONE'] = all_shots['SHOT_ZONE_BASIC'] + " - " + all_shots['SHOT_ZONE_AREA'] + " - " + all_shots['SHOT_ZONE_RANGE']
all_shots = all_shots.drop(columns=["SHOT_ZONE_BASIC","SHOT_ZONE_AREA","SHOT_ZONE_RANGE"], errors = 'ignore')

# remove backcourt, not worth analysing
all_shots = all_shots[~all_shots["SHOT_ZONE"].str.contains("Back Court")]
all_shots = all_shots[~all_shots["SHOT_ZONE"].str.contains("Backcourt")]

# Optional: check remaining zones
all_shots["SHOT_ZONE"].value_counts()

all_shots["SHOT_ZONE"].value_counts()

SHOT_ZONE
Restricted Area - Center(C) - Less Than 8 ft.          66762
In The Paint (Non-RA) - Center(C) - Less Than 8 ft.    19173
Above the Break 3 - Left Side Center(LC) - 24+ ft.     16458
Above the Break 3 - Right Side Center(RC) - 24+ ft.    15660
Above the Break 3 - Center(C) - 24+ ft.                10785
Mid-Range - Right Side Center(RC) - 16-24 ft.           8028
Left Corner 3 - Left Side(L) - 24+ ft.                  7759
Mid-Range - Left Side Center(LC) - 16-24 ft.            7635
Mid-Range - Right Side(R) - 8-16 ft.                    7552
Mid-Range - Left Side(L) - 8-16 ft.                     7544
Right Corner 3 - Right Side(R) - 24+ ft.                7276
Mid-Range - Center(C) - 16-24 ft.                       7214
In The Paint (Non-RA) - Center(C) - 8-16 ft.            6756
Mid-Range - Left Side(L) - 16-24 ft.                    5383
Mid-Range - Right Side(R) - 16-24 ft.                   4869
Mid-Range - Center(C) - 8-16 ft.                        2228
In The Paint (

In [149]:
all_shots["ACTION_TYPE"].value_counts()

ACTION_TYPE
Jump Shot                             96966
Layup Shot                            17988
Driving Layup Shot                    12887
Pullup Jump shot                      12102
Floating Jump shot                     5175
Hook Shot                              4627
Step Back Jump shot                    4414
Tip Layup Shot                         4178
Running Layup Shot                     3563
Turnaround Jump Shot                   3548
Dunk Shot                              3246
Cutting Layup Shot                     3225
Fadeaway Jump Shot                     2934
Driving Finger Roll Layup Shot         2194
Driving Floating Jump Shot             2183
Putback Layup Shot                     1955
Reverse Layup Shot                     1911
Running Jump Shot                      1786
Turnaround Hook Shot                   1763
Jump Bank Shot                         1757
Turnaround Fadeaway shot               1633
Cutting Dunk Shot                      1494
Alley Oop Dunk Shot 

In [150]:
# Categorize Action Types
jump_shots = ["Jump Shot", "Running Jump Shot", "Pullup Jump shot", "Step Back Jump shot",
              "Floating Jump shot", "Turnaround Jump Shot", "Running Pull-Up Jump Shot",
              "Turnaround Fadeaway Bank Jump Shot", "Driving Jump shot", "Turnaround Fadeaway shot",
              "Fadeaway Jump Shot", "Driving Floating Jump Shot", "Turnaround Fadeaway Bank Jump Shot"]

bank_shots = ["Jump Bank Shot", "Turnaround Bank shot", "Driving Floating Bank Jump Shot",
              "Step Back Bank Jump Shot", "Fadeaway Bank shot", "Hook Bank Shot", "Driving Bank shot",
             "Pullup Bank shot"]

layups = ["Layup Shot", "Driving Layup Shot", "Running Layup Shot", "Cutting Layup Shot",
          "Finger Roll Layup Shot", "Putback Layup Shot", "Alley Oop Layup shot",
          "Driving Reverse Layup Shot", "Running Finger Roll Layup Shot", "Running Alley Oop Layup Shot",
          "Running Reverse Layup Shot", "Reverse Layup Shot", "Driving Finger Roll Layup Shot",
          "Tip Layup Shot", "Cutting Finger Roll Layup Shot"]

dunks = ["Dunk Shot", "Driving Dunk Shot", "Putback Dunk Shot", "Alley Oop Dunk Shot",
         "Running Dunk Shot", "Reverse Dunk Shot", "Tip Dunk Shot", "Running Alley Oop Dunk Shot",
         "Running Reverse Dunk Shot", "Cutting Dunk Shot", "Driving Reverse Dunk Shot"]

hooks = ["Driving Hook Shot", "Turnaround Hook Shot", "Turnaround Bank Hook Shot", "Driving Bank Hook Shot",
         "Running Hook Shot", "Hook Shot", "Hook Bank Shot"]

# Map function
def categorize_shot(action):
    if action in jump_shots:
        return "Jump Shot"
    elif action in bank_shots:
        return "Bank Shot"
    elif action in layups:
        return "Layup"
    elif action in dunks:
        return "Dunk"
    elif action in hooks:
        return "Hook Shot"
    else:
        return "Other"


In [151]:
all_shots["SHOT_CATEGORY"] = all_shots["ACTION_TYPE"].apply(categorize_shot)
all_shots = all_shots.drop(columns=["ACTION_TYPE"], errors = 'ignore')

all_shots = all_shots[~all_shots["SHOT_CATEGORY"].str.contains("Other")]

In [152]:
all_shots["SHOT_CATEGORY"].value_counts()

SHOT_CATEGORY
Jump Shot    131144
Layup         51996
Dunk          10083
Hook Shot      7566
Bank Shot      4241
Name: count, dtype: int64

In [153]:
rows_before = all_shots.shape[0]

# Drop rows with any missing values
all_shots = all_shots.dropna()

# Count rows after dropping
rows_after = all_shots.shape[0]

# Report how many rows were removed
rows_removed = rows_before - rows_after
print(f"Rows removed due to missing values: {rows_removed}")

Rows removed due to missing values: 0


In [154]:
all_shots

,PLAYER_NAME,TEAM_NAME,PERIOD,SHOT_MADE_FLAG,SHOT_VALUE,TIME_LEFT_SEC,SHOT_ZONE,SHOT_CATEGORY
0,Wesley Matthews,Dallas Mavericks,1,0,3,544,Above the Break 3 - Right Side Center(RC) - 24...,Jump Shot
1,Wesley Matthews,Dallas Mavericks,2,0,3,662,Above the Break 3 - Left Side Center(LC) - 24+...,Jump Shot
2,Wesley Matthews,Dallas Mavericks,2,1,3,490,Above the Break 3 - Right Side Center(RC) - 24...,Jump Shot
3,Wesley Matthews,Dallas Mavericks,3,1,3,657,Above the Break 3 - Right Side Center(RC) - 24...,Jump Shot
4,Wesley Matthews,Dallas Mavericks,3,0,2,573,Mid-Range - Right Side(R) - 16-24 ft.,Jump Shot
...,...,...,...,...,...,...,...,...
205533,Markieff Morris,Washington Wizards,1,1,2,264,Restricted Area - Center(C) - Less Than 8 ft.,Layup
205534,Markieff Morris,Washington Wizards,1,1,2,200,In The Paint (Non-RA) - Center(C) - 8-16 ft.,Jump Shot
205535,Markieff Morris,Washington Wizards,3,0,3,618,Above the Break 3 - Center(C) - 24+ ft.,Jump Shot
205536,Markieff Morris,Washington Wizards,3,1,2,493,Mid-Range - Center(C) - 16-24 ft.,Jump Shot


In [155]:
all_shots.to_csv("all_shots_clean.csv", index=False) # CLEANED DATASET

In [121]:
nba_teams = [
    # Eastern Conference
    "Boston Celtics", "Brooklyn Nets", "New York Knicks", "Philadelphia 76ers", "Toronto Raptors",
    "Chicago Bulls", "Cleveland Cavaliers", "Detroit Pistons", "Indiana Pacers", "Milwaukee Bucks",
    "Atlanta Hawks", "Charlotte Hornets", "Miami Heat", "Orlando Magic", "Washington Wizards",
    
    # Western Conference
    "Denver Nuggets", "Minnesota Timberwolves", "Oklahoma City Thunder", "Portland Trail Blazers", "Utah Jazz",
    "Golden State Warriors", "Los Angeles Clippers", "Los Angeles Lakers", "Phoenix Suns", "Sacramento Kings",
    "Dallas Mavericks", "Houston Rockets", "Memphis Grizzlies", "New Orleans Pelicans", "San Antonio Spurs"
]


In [122]:
team_dfs = {team: all_shots[all_shots["TEAM_NAME"] == team].copy() for team in nba_teams}

In [123]:
team_dfs["Boston Celtics"] # example ofimport pandas as pd


,PLAYER_NAME,TEAM_NAME,PERIOD,SHOT_MADE_FLAG,SHOT_VALUE,TIME_LEFT_SEC,SHOT_ZONE,SHOT_CATEGORY
24703,Jared Sullinger,Boston Celtics,1,0,3,270,Above the Break 3 - Left Side Center(LC) - 24+...,Jump Shot
24704,Jared Sullinger,Boston Celtics,1,1,2,83,Restricted Area - Center(C) - Less Than 8 ft.,Layup
24705,Jared Sullinger,Boston Celtics,1,1,2,3,In The Paint (Non-RA) - Center(C) - Less Than ...,Jump Shot
24706,Jared Sullinger,Boston Celtics,2,1,2,698,Restricted Area - Center(C) - Less Than 8 ft.,Layup
24707,Jared Sullinger,Boston Celtics,3,1,2,636,Restricted Area - Center(C) - Less Than 8 ft.,Layup
...,...,...,...,...,...,...,...,...
201126,Terry Rozier,Boston Celtics,4,1,2,310,In The Paint (Non-RA) - Center(C) - Less Than ...,Jump Shot
201127,Terry Rozier,Boston Celtics,4,1,2,277,Restricted Area - Center(C) - Less Than 8 ft.,Layup
201128,Terry Rozier,Boston Celtics,4,0,2,224,Mid-Range - Left Side(L) - 16-24 ft.,Jump Shot
201129,Terry Rozier,Boston Celtics,2,0,2,146,Restricted Area - Center(C) - Less Than 8 ft.,Layup


In [124]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Load dataset
df = pd.read_csv("all_shots_clean.csv")

# Normalize column names
df.columns = df.columns.str.strip().str.replace(' ', '_')

# Teams
teams = df['TEAM_NAME'].unique()

# Store results
results = []

for team in teams:
    team_df = df[df['TEAM_NAME'] == team].copy()
    
    # Features and target
    X = team_df[['PLAYER_NAME', 'PERIOD', 'SHOT_VALUE', 'TIME_LEFT_SEC', 'SHOT_ZONE', 'SHOT_CATEGORY']].copy()
    y = team_df['SHOT_MADE_FLAG']
    
    # Label encode categorical features
    for col in ['PLAYER_NAME', 'SHOT_ZONE', 'SHOT_CATEGORY']:
        X[col] = LabelEncoder().fit_transform(X[col])
    
    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # XGBoost
    xgb = XGBClassifier(n_estimators=50, max_depth=5, eval_metric='logloss', random_state=42)
    xgb.fit(X_train, y_train)
    xgb_acc = accuracy_score(y_test, xgb.predict(X_test))
    
    # Random Forest
    rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    rf.fit(X_train, y_train)
    rf_acc = accuracy_score(y_test, rf.predict(X_test))

    
    # Store results with average and range
    acc_list = [xgb_acc, rf_acc, lr_acc]
    results.append({
        'TEAM_NAME': team,
        'XGBoost_ACC': xgb_acc,
        'RandomForest_ACC': rf_acc,
        'AVG_ACC': sum(acc_list)/len(acc_list),
        'RANGE_ACC': max(acc_list) - min(acc_list)
    })

# Convert to DataFrame
final_table = pd.DataFrame(results).sort_values(by='AVG_ACC', ascending=False)

# Show final table
final_table

,TEAM_NAME,XGBoost_ACC,RandomForest_ACC,AVG_ACC,RANGE_ACC
28,Washington Wizards,0.641723,0.647770,0.608394,0.112081
27,Los Angeles Lakers,0.624549,0.633935,0.598058,0.098246
4,Philadelphia 76ers,0.621045,0.629875,0.595536,0.094186
2,Phoenix Suns,0.618978,0.627007,0.593891,0.091318
10,Houston Rockets,0.609685,0.634629,0.593334,0.098940
0,Dallas Mavericks,0.618459,0.625000,0.593049,0.089311
26,Boston Celtics,0.615753,0.627397,0.592947,0.091708
14,Indiana Pacers,0.612787,0.629310,0.592596,0.093621
5,Cleveland Cavaliers,0.621960,0.616065,0.591238,0.086271
25,Miami Heat,0.603236,0.633282,0.590736,0.097593


In [125]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Load dataset
df = pd.read_csv("all_shots_clean.csv")

# Normalize column names
df.columns = df.columns.str.strip().str.replace(' ', '_')

# Teams
teams = df['TEAM_NAME'].unique()

# Store results
results = []

for team in teams:
    team_df = df[df['TEAM_NAME'] == team].copy()
    
    # Features and target
    X = team_df[['PLAYER_NAME', 'PERIOD', 'SHOT_VALUE', 'TIME_LEFT_SEC', 'SHOT_ZONE', 'SHOT_CATEGORY']].copy()
    y = team_df['SHOT_MADE_FLAG']
    
    # Label encode categorical features
    for col in ['PLAYER_NAME', 'SHOT_ZONE', 'SHOT_CATEGORY']:
        X[col] = LabelEncoder().fit_transform(X[col])
    
    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # XGBoost
    xgb = XGBClassifier(n_estimators=50, max_depth=5, eval_metric='logloss', random_state=42)
    xgb.fit(X_train, y_train)
    xgb_acc = accuracy_score(y_test, xgb.predict(X_test))
    
    # Random Forest
    rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    rf.fit(X_train, y_train)
    rf_acc = accuracy_score(y_test, rf.predict(X_test))
    
    # Store results
    results.append({
        'TEAM_NAME': team,
        'XGBoost_ACC': xgb_acc,
        'RandomForest_ACC': rf_acc
    })

# Convert to DataFrame
accuracy_df = pd.DataFrame(results).sort_values(by='XGBoost_ACC', ascending=False)

# Print table of accuracies per team
print("=== Accuracy per Team ===")
print(accuracy_df)

# Compute average accuracy across the league for each model
avg_xgb = accuracy_df['XGBoost_ACC'].mean()
avg_rf = accuracy_df['RandomForest_ACC'].mean()

print("\n=== Average Accuracy Across All Teams ===")
print(f"XGBoost Average Accuracy: {avg_xgb:.3f}")
print(f"Random Forest Average Accuracy: {avg_rf:.3f}")


=== Accuracy per Team ===
                 TEAM_NAME  XGBoost_ACC  RandomForest_ACC
28      Washington Wizards     0.641723          0.647770
27      Los Angeles Lakers     0.624549          0.633935
5      Cleveland Cavaliers     0.621960          0.616065
4       Philadelphia 76ers     0.621045          0.629875
2             Phoenix Suns     0.618978          0.627007
0         Dallas Mavericks     0.618459          0.625000
26          Boston Celtics     0.615753          0.627397
21        Sacramento Kings     0.615004          0.617127
14          Indiana Pacers     0.612787          0.629310
17          Denver Nuggets     0.610549          0.624277
10         Houston Rockets     0.609685          0.634629
18               Utah Jazz     0.608129          0.624233
29   Oklahoma City Thunder     0.605654          0.614134
25              Miami Heat     0.603236          0.633282
1   Minnesota Timberwolves     0.602853          0.609610
7            Atlanta Hawks     0.602750       

In [126]:
# random forest better for generalized shot selection, 

In [127]:
# particulr shot selection captured better by xg_boost?

In [128]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Load dataset
df = allTypes

# Normalize column names
df.columns = df.columns.str.strip().str.replace(' ', '_')

# Teams
teams = df['TEAM_NAME'].unique()

# Store results
results = []

for team in teams:
    team_df = df[df['TEAM_NAME'] == team].copy()
    
    # Features and target
    X = team_df[['PLAYER_NAME', 'PERIOD', 'SHOT_VALUE', 'TIME_LEFT_SEC', 'SHOT_ZONE_BASIC','SHOT_ZONE_AREA','SHOT_ZONE_RANGE', 'ACTION_TYPE']].copy()
    y = team_df['SHOT_MADE_FLAG']
    
    # Label encode categorical features
    for col in ['PLAYER_NAME', 'SHOT_ZONE_BASIC','SHOT_ZONE_AREA','SHOT_ZONE_RANGE', 'ACTION_TYPE']:
        X[col] = LabelEncoder().fit_transform(X[col])
    
    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # XGBoost
    xgb = XGBClassifier(n_estimators=50, max_depth=5, eval_metric='logloss', random_state=42)
    xgb.fit(X_train, y_train)
    xgb_acc = accuracy_score(y_test, xgb.predict(X_test))
    
    # Random Forest
    rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    rf.fit(X_train, y_train)
    rf_acc = accuracy_score(y_test, rf.predict(X_test))
    
    # Store results
    results.append({
        'TEAM_NAME': team,
        'XGBoost_ACC': xgb_acc,
        'RandomForest_ACC': rf_acc
    })

# Convert to DataFrame
accuracy_df = pd.DataFrame(results).sort_values(by='XGBoost_ACC', ascending=False)

# Print table of accuracies per team
print("=== Accuracy per Team ===")
print(accuracy_df)

# Compute average accuracy across the league for each model
avg_xgb = accuracy_df['XGBoost_ACC'].mean()
avg_rf = accuracy_df['RandomForest_ACC'].mean()

print("\n=== Average Accuracy Across All Teams ===")
print(f"XGBoost Average Accuracy: {avg_xgb:.3f}")
print(f"Random Forest Average Accuracy: {avg_rf:.3f}")

=== Accuracy per Team ===
                 TEAM_NAME  XGBoost_ACC  RandomForest_ACC
28      Washington Wizards     0.714178          0.727753
4       Philadelphia 76ers     0.690896          0.687225
27      Los Angeles Lakers     0.679856          0.685612
3    Golden State Warriors     0.675978          0.675279
16         Milwaukee Bucks     0.660915          0.673668
15    New Orleans Pelicans     0.647228          0.666667
22         Detroit Pistons     0.643816          0.640283
21        Sacramento Kings     0.640790          0.649259
18               Utah Jazz     0.640183          0.650879
29   Oklahoma City Thunder     0.639831          0.639124
11             LA Clippers     0.639405          0.647584
1   Minnesota Timberwolves     0.637931          0.635682
26          Boston Celtics     0.632262          0.653452
12       Memphis Grizzlies     0.629540          0.629540
19       Charlotte Hornets     0.628159          0.644043
7            Atlanta Hawks     0.625723       

In [129]:
#accuracy improved a ton by adding distinct shots 

In [157]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Load dataset
df = all_shots_renamed

# Normalize column names
df.columns = df.columns.str.strip().str.replace(' ', '_')

# Define features and target
categorical_cols = [
    'PLAYER_NAME', 'TEAM_NAME', 'ACTION_TYPE', 'HTM', 'VTM',
    'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE'
]
numeric_cols = [
    'PERIOD', 'SHOT_DISTANCE', 'LOC_X', 'LOC_Y', 
     'SHOT_VALUE', 'TIME_LEFT_SEC'
]
target_col = 'SHOT_MADE_FLAG'

# Encode categorical features
df_encoded = df.copy()
for col in categorical_cols:
    df_encoded[col] = LabelEncoder().fit_transform(df_encoded[col])

# Features and target
X = df_encoded[categorical_cols + numeric_cols]
y = df_encoded[target_col]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ----- Random Forest -----
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf.fit(X_train, y_train)
rf_acc = accuracy_score(y_test, rf.predict(X_test))

# ----- XGBoost -----
xgb = XGBClassifier(n_estimators=50, max_depth=5, eval_metric='logloss', random_state=42)
xgb.fit(X_train, y_train)
xgb_acc = accuracy_score(y_test, xgb.predict(X_test))

# Print results
print("Random Forest Accuracy:", round(rf_acc, 3))
print("XGBoost Accuracy:", round(xgb_acc, 3))


Random Forest Accuracy: 0.653
XGBoost Accuracy: 0.661
